In [ ]:
import csv
import io
import os
from pathlib import Path

from dotenv import load_dotenv
from O365 import Account
from tqdm import tqdm

load_dotenv(Path("~").expanduser() / "O365.env")

In [ ]:
def getO365Client():
    app_principal = {
        "client_id": os.environ["O365_client_id"],
        "client_secret": os.environ["O365_client_secret"],
        "tenant_id": os.environ["O365_tenant_id"],
    }
    auth = (app_principal["client_id"], app_principal["client_secret"])
    account = Account(
        auth, auth_flow_type="credentials", tenant_id=app_principal["tenant_id"]
    )
    if account.authenticate(scope=[]):
        return account
    return None


account = getO365Client()

In [ ]:
site = account.sharepoint().search_site("Purchasing Bids")[0]

In [ ]:
library = site.list_document_libraries()[0]
library

In [ ]:
root = list(library.get_items())
root.reverse()

In [ ]:
allowed = [
    "Purchasing Bids Owners",
    "Purchasing Bids Visitors",
    "Purchasing Bids Members",
    None,
]
yearsToCheck = 3
print(f"scanning {yearsToCheck} years")
permissionsToRemove = []
for folder in root:
    if yearsToCheck == 0:
        break
    if "RFPs" in folder.name:
        items = list(folder.get_items())
        for subfolder in tqdm(items, desc=folder.name):
            for permission in subfolder.get_permissions():
                if permission.granted_to not in allowed:
                    permissionsToRemove.append(permission)

        for permission in folder.get_permissions():
            if permission.granted_to not in allowed:
                permissionsToRemove.append(permission)
        yearsToCheck -= 1

In [ ]:
print("Exporting Report")
with open("report.csv", "w") as csvfile:
    fieldnames = ["folder", "file name", "permission", "users name"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for p in permissionsToRemove:
        if p.roles[0] == "read":
            continue
        writer.writerow(
            {
                "folder": p._parent._parent.name,
                "file name": p._parent.name,
                "permission": p.roles[0],
                "users name": p.granted_to,
            }
        )

In [ ]:
permissionsToRemove